# Notes about this lecture
Because of the Corona virus outbreak, this lecture will not be held in the classroom but online only. Further, the lecture will only be available in this written form. In order to offer support for the students we will use the gitlab issue tracker as a question & answer forum: https://git.ee.ethz.ch/python-for-engineers/class-fs20-forum and individual videoconference sessions when needed.

## Software

### Necessary software
Please install the following tools:
* python3 (https://www.python.org/downloads/ version 3.8.2 is fine.
Python is a prerequisite for jupyter)
* jupyter-notebook (https://jupyter.org/install.html)
* **Hint for Windows and OSX**: Try to install conda or miniconda (https://docs.conda.io/en/latest/miniconda.html) first. This will install Python and jupyter-notebook automatically.

### Optional (but highly recommended) software
* git (https://git-scm.com/download/). Git is harder to install but not strictly necessary. **Hint**: On Windows Git will automatically install a Linux compatible shell which can then be found as 'Git BASH'.
* If git is not available, solutions shall be uploaded on https://polybox.ethz.ch instead and the folder shall be shared with the lecturers. 

## Support
**For any issues please use the forum** at: https://git.ee.ethz.ch/python-for-engineers/class-fs20-forum and follow the instructions therein. In case of need, we will open a room using Jitsi or BigBlueButton and share the audio, video or the screen: make sure you have a microphone and speakers functioning. 

This service is offered only **during the normal lecture hours**.

# Obtaining the material for this lecture
### If git is available on your system (preferred option)
Pull the new material from the upstream repository:

```bash
cd class-fs20
git pull upstream master
```

Then launch the jupyter-notebook and open the Lecture_XX file:

```bash
anaconda # Only on ETH computers to load the Python environment.
jupyter-notebook &
```

### If git is **not** available on your system
Download the latest material from:
https://git.ee.ethz.ch/python-for-engineers/class-fs20/-/archive/master/class-fs20-master.zip
and unpack it on your computer.

# Announcement:  Open "Hilfsassistenten" position for upcoming P&S class

We are looking for "Hilfsassistenten" for the next edition for this Python P&S. If you enjoyed the lecture and you are interested to help us out in the next semester, please write us an email!

# Refreshing previous lectures

Please open the jupyter-notebook of the past lecture and read through it. This will help fixing the learned notions into the long-term memory.

# Lecture 13: Selected topics about cracking with Python

## A note on the definition of the words *hacking* and *cracking*

Quoting Richard Stallman, "*it is hard to write a simple definition of something as varied as hacking, but I think what these activities have in common is playfulness, cleverness, and exploration. Thus, hacking means exploring the limits of what is possible, in a spirit of playful cleverness. Activities that display playful cleverness have "hack value".*" https://stallman.org/articles/on-hacking.html.

According to this definition, using three chopsticks instead of two for having fun during a dinner would be a form of *hacking* for example.

Quoting again https://stallman.org/articles/on-hacking.html: "*Around 1980, when the news media took notice of hackers, they fixated on one narrow aspect of real hacking: the security breaking which some hackers occasionally did. They ignored all the rest of hacking, and took the term to mean breaking security, no more and no less. The media have since spread that definition, disregarding our attempts to correct them. As a result, most people have a mistaken idea of what we hackers actually do and what we think.*" 

The Cambridge Dictionary reports a definition not too far from the one of Stallman:

**hacker:** *a person who is skilled in the use of computer systems, often one who illegally obtains access to private computer systems* https://dictionary.cambridge.org/dictionary/english/hacker

While the [fake?] Oxford *living* dictionary reports just the narrow meaning of security breaking: 

**hacking:** *the gaining of unauthorized access to data in a system or computer.* https://en.oxforddictionaries.com/definition/hacking

Quoting again Stallman: "*You can help correct the misunderstanding simply by making a distinction between security breaking and hacking—by using the term "cracking" for security breaking. The people who do it are "crackers". Some of them may also be hackers, just as some of them may be chess players or golfers; most of them are not.*"

## Analysing SQLite databases of Firefox 
We will begin the lecture with a simple example of *digital forensics*, which is the science of recovering and investigating material found on digital devices https://en.wikipedia.org/wiki/Digital_forensics. 

Such techniques are sometimes used by attackers to gather knowledge on their victims, and attack them with *social engineering* https://en.wikipedia.org/wiki/Social_engineering_(security) techniques.

The following example reads some data stored in some SQL databases of Firefox. For more information about the Firefox databases, visit https://developer.mozilla.org/en-US/docs/Mozilla/Tech/Places/Database .

In [12]:
import sqlite3

def print_places(database):
    print('Printing places:', database)
    connection = sqlite3.connect(database)
    cursor = connection.cursor()
    cursor.execute('SELECT url FROM moz_places;')
    for row in cursor:
        print(row[0])
        
def print_historyvisits(database):
    print('Printing hitoryvisits:', database)
    connection = sqlite3.connect(database)
    cursor = connection.cursor()
    cursor.execute('SELECT id, from_visit, visit_date FROM moz_historyvisits;')
    for row in cursor:
        print(row[0], row[1])

# Use unix-style path name pattern extension to find the 'places.sqlite' databases.
from os.path import expanduser
import glob
homedir = expanduser("~")
paths = glob.glob(homedir + "/.mozilla/firefox/*.*/places.sqlite")

# Alternatively, enter the path manually.
paths = ["Users\mathi\AppData\Roaming\Mozilla\Firefox\Profiles\nuhriviq.default-release\places.sqlite"]  # Replace with your own path.
#found the file but doesn t want to be opened...
print(paths)

if len(paths) == 0:
    print("No path could be found. Enter the path manually!")

# Extract the browsing history for each database found.
for path in paths:
    try:
        print_places(path)
        print_historyvisits(path)
    except Exception as e:
        print("Error:", e)
        
# An error "database is locked" means that Firefox is currently running 
# and the database should not be accessed by another program.

['Users\\mathi\\AppData\\Roaming\\Mozilla\\Firefox\\Profiles\nuhriviq.default-release\\places.sqlite']
Printing places: Users\mathi\AppData\Roaming\Mozilla\Firefox\Profiles
uhriviq.default-release\places.sqlite
Error: unable to open database file


## Micro exercise 1
Run the above code and adapt it as you wish. If no `paths` can be found you may have to set the paths manually by replacing the XXXXX with the correct values or by entering a totally different path to the `places.sqlite` database.

To simplify the seach task, it is possible to run in a terminal:

```find . -name 'places.sqlite'```

**For your privacy: restart the kernel after having run this exercise and do not save your personal data in this notebook.**

## Cracking passwords
We will start with cracking a file residing on our hard-drive for avoiding the problems related to the low bandwidth of the network.

## Brute-force cracking a password-protected zip file

We will look at two methods: 
1. scanning at all possible combination of passwords
1. using a dictionary (downloadable from the internet) of common passwords

### The ```zipfile``` module
The ```zipfile``` module provides a convenient interface to zip files. The method ```extractall``` allows to open a password-protected file, and it returns error if the passwords is *not* correct. 

The example below shows the basic functionality of this method. 

First, however, let's compress and encrypt a file uzing zip. 

In a terminal:

```zip --encrypt crackme.txt.zip crackme.txt ```

If you have troubles encrypting a zip file on your system, you can use the file `crackme.txt.zip` which is already present in your folder, and which is encrypted with the password `zzz`.

In [16]:
# Let's check if it works:
import zipfile

def crack(file, my_pwd):
    try:
        file.extractall(pwd=str.encode(my_pwd))
        print('Success! The password is: ', my_pwd)
    except:
        print('Wrong password.')
    
file = zipfile.ZipFile('crackme.txt.zip')
crack(file, 'zzz')


Success! The password is:  zzz


### Scanning all possible passwords of a given length (slow but complete)
The module ```itertools``` and ```string``` come to help in this task. The module ```itertools``` provides the method ```product()``` which creates all possible permutations of a given length (set by the parameter ```repeat```) of some predefined symbols.

As predefined symbols we take all possible characters, including letters, numbers and special characters. There is no need to write them by hand, since the module ```string``` provides them with the calls ```string.ascii_letters```, ```string.digits``` and ```string.punctuation``` respectively.

In [17]:
# It takes 0.63 seconds to scan all 2-letters combinations.
# It takes 53 seconds to scan all 3-letters combinations.
# It takes 15 seconds to find 'zzz' passwords when scanning 3-letters pwds.
# Notice: It is possible to speed up the search by parallelizing it.
import zipfile
import string
import itertools
import time


def crack(file, my_pwd):
    try:
        #print('tryint this: ', my_pwd)
        file.extractall(pwd=str.encode(my_pwd))
        print('Success! The password is: ', my_pwd)
        end_time = time.time()
        print('Done. Time used [s]: ', str(end_time -start_time))
    except:
        #print('Wrong password.')
        pass
    
file = zipfile.ZipFile('crackme.txt.zip')
my_letters = string.ascii_letters + string.digits + string.punctuation
print(my_letters)

start_time = time.time()

for permutation in map(''.join, itertools.product(my_letters, repeat = 3)):
    crack(file, permutation)
    
    
end_time = time.time()
print('No password found. Time used [s]: ', str(end_time - start_time))

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
Success! The password is:  zzz
Done. Time used [s]:  42.393322467803955
No password found. Time used [s]:  164.5825686454773


### Scanning the most common passwords
By analysing several leaks, people on the internet have compiled several databases of most common passwords. Some of these are:


1. John the Ripper (20kB) http://downloads.skullsecurity.org/passwords/john.txt.bz2
1. SVNDigger (2MB)
https://www.netsparker.com/s/research/SVNDigger.zip
1. DirBuster-Project (38MB)
http://downloads.sourceforge.net/dirbuster/DirBuster-Lists.tar.bz
1. Breach Compilation (44GB). On December 11 2018 there has been a leak of 1.4 billion email/password pairs. These have been circulating freely on the internet and can be downloaded for example here: ** magnet:?xt=urn:btih:5a9ba318a5478769ddc7393f1e4ac928d9aa4a71&dn=breachcompilation.txt.7z ** (in Debian: launch "Transmission" then "File" --> "Open URL")


You can download them on your own. For convenience, a copy of the two smallest databases is provided in ```Lecture_13/john.txt``` and ```Lecture_13/SVNDigger```.

In [20]:
# It takes 1.2 seconds to check all passwords in John the Ripper
# It takes 20 seconds to check all passwords in 'SVNDigger/all-extensionless.txt'
import zipfile
import string
import time


def crack(file, my_pwd):
    try:
        #print('tryint this: ', my_pwd)
        file.extractall(pwd=str.encode(my_pwd))
        print('Success! The password is: ', my_pwd)
        end_time = time.time()
        print('Done. Time used [s]: ', str(end_time - start_time))
    except:
        #print('Wrong password.')
        pass
    
file = zipfile.ZipFile('crackme.txt.zip')


# reading passwords from the database:
#with open('john.txt') as f:
with open('SVNDigger/all-extensionless.txt') as f:
    read_data = f.read()   
    read_data = read_data.split('\n')
    len(read_data)
    
print('The database contains so many elements: ', len(read_data))
#print(read_data)

start_time = time.time()

for password in read_data:
    crack(file, password)
    
    
end_time = time.time()
print('No password found. Time used [s]: ', str(end_time - start_time))


The database contains so many elements:  25420
No password found. Time used [s]:  6.162144899368286


# Mastering `http` with Python: the `requests` module

Is it possible to answer an online poll using a Python script?

Is it possible to login to a website with Python?

The module `requests` comes to help.

Documentation: https://2.python-requests.org/

### The HyperText Transfer Protocol

The HyperText Transfer Protocol (HTTP) today is everywhere. Common web browsers use almost exclusively HTTP for visiting web pages. Even most mobile phone "apps" communicate to their home-servers using HTTP. 

When using HTTP, the client and server communicate by sending plain-text messages. The client sends *requests* to the server and the server sends *responses*. Here is a minimal example of the *request* that browsers send to the ETH web server when visiting the page www.ethz.ch:

```
GET / HTTP/1.1
Host: www.ethz.ch
```

#### Response Status Codes

In each response, there is a HTTP status code which indicates the type of result. The status code is `200` in most cases: it means no error, everything OK. The famous `404` status code indicates that no resource was found at the requested URL. See [Wikipedia](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes) for a full list of defined status codes.

#### Request Methods

HTTP defines methods to indicate the desired action to be performed. There are seven methods in total:
1. **GET**
1. **POST**
1. PUT
1. HEAD
1. DELETE
1. PATCH
1. OPTIONS

with GET and POST being the most commonly used. The GET method is used to retrieve information, POST is used to send information to the web server, e.g. when you entered your shipping address into an online shop form. The complete and official documentation is available here: https://www.w3schools.com/tags/ref_httpmethods.asp

The ```requests``` module has methods which corresponds exactly to the HTTP methods, as exemplified below.

For more information visit the official documentation on 
https://2.python-requests.org/ . Notice that there is a new version (III) at 
https://3.python-requests.org/ but which is not installed on the `tardis` machines.

### A simple example of GET: low-level socket vs. ```requests```
In the past lectures, we have already seen the code below which sends a GET request to the www.ethz.ch web page. The ```socket``` module gives low-level access to the ethernet interface, and of course everything that the ```requests``` module does could be done with the ```socket``` module instead.

In [21]:
import socket

# Create a client socket and connect to the server.
# AF = Address Family
# INET = IPv4
# SOCK_STREAM = TCP
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("ethz.ch", 80))

# Send an 'HTTP GET' request to the server.
message_to_send = "GET /index.html HTTP/1.1\r\nHost: ethz.ch\r\n\r\n"
# Encode the message as bytes.
message_bytes = message_to_send.encode('utf-8')
s.sendall(message_bytes) # Send the bytes.

# Receive at most 4096 bytes of the response.
response = s.recv(4096)

print("Server response:")
# The expected response is an HTTP header telling
# us that the page has moved to https://ethz.ch/index.html.
print(response.decode('utf-8'))

# The response should be an HTTP header.

Server response:
HTTP/1.1 301 
Date: Tue, 26 May 2020 11:43:53 GMT
Server: Varnish
X-Varnish: 90933817
location: https://ethz.ch/index.html
Content-Length: 0
Connection: keep-alive




This is instead how the same code above looks like when using the ```requests``` module:

In [22]:
import requests

target_host = 'http://www.ethz.ch'  # Notice the "http://" added to the socket example.

response = requests.get(target_host)

print('Response url: ', response.url)  # Print just the "url" part of the response
print('Response status code: ', response.status_code)  
print('All available attributes: ', dir(response)) 

Response url:  https://www.ethz.ch/
Response status code:  404
All available attributes:  ['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']


### Firefox tools to analyse web pages
Firefox offers the following tools which are very practical for examining web pages:
1. ***Inspector***: right click on an item in a web page, then click *Inspect Element*. A window opens at the bottom of the screen highlighting the HTML code part which is responsible of the selected item. The Inspector can be launched also with *Tools --> Web Developer --> Inspector* from the top menu (press the ```Alt``` key to visualize it).
1. ***Network (Inspector)*** In the Inspector window there is a *Network* option which allows to follow the HTTP exchange as items are clicked in the page

### Micro exercise 2
Open the site https://dudle.inf.tu-dresden.de/python-class-123/ with Firefox. Then place the mouse on the white entry box, right-click, and choose "Inspect Element". An extra window appears on the bottom showing the HTML code of the site. From the highlighted text we learn that the entry element has the property `name=add_participant` (which will be useful later on).

If we scroll the code slightly up, we find that the entry element belongs to a *form* of type *POST*. In fact, we find the text `<form method="post" action="." accept-charset="utf-8">`.

If we now click one of the green, red or orange radio buttons and inspect these elements, we find their related names (such as `add_participant_checked_blue` when inspecting the `blue` column) and values (such as `a_yes__`, `c_no__` or `b_maybe`).

Run therefore the following example which runs the ```post``` method of the module ```requests``` to vote on the dudle:

In [24]:
# NOTICE: RUN MAXIMUM TWO TIMES THIS CODE! Otherwise the admin
# of the server will notice strange traffic and block the page (it
# aldready happened).
#
# Vote on a dudle using "requests" and three lines of python code.
import requests

# Replace 'Python student 1' with your nickname.
payload = {'add_participant': 'fOlLoW_mY_iNStaGrAm_:@defonotme', 'add_participant_checked_blue' : 'a_no__' , 'add_participant_checked_green' : 'a_no__'}

'''
Notice that using the Firefox Inspector the entry reveals to be:
<form method="post" action=....>  so: notice the POST!
This means that it should be used 
requests.posts, and not requests.get or requests.put.
'''
response = requests.post('https://dudle.inf.tu-dresden.de/python-class-123/', data = payload)

# Printing the response from the server. If successful, it returns "200":
print(response)

print("Refresh the html page to see if the vote has been registered correctly. Use a different participant name if necessary.")

<Response [200]>
Refresh the html page to see if the vote has been registered correctly. Use a different participant name if necessary.


## A slightly more powerful module, but less low-level: RoboBrowser
The same functionality described above can be achieved utilizing the module ```robobrowser```. To install this module and its dependence try with `pip install robobrowser`, `pip install bs4`, `pip install werkzeug`).

The robobroser module does not require to understand the HTML language as the ```requests``` module, and it can handle, for example, form types automatically as shown below:

In [26]:
# NOTICE: RUN MAXIMUM ONE TIME THIS CODE!
#
import werkzeug
werkzeug.cached_property = werkzeug.utils.cached_property  # Fix a bug in werkzeug.
from robobrowser import RoboBrowser

browser = RoboBrowser()
browser.open('https://dudle.inf.tu-dresden.de/python-class-123/')

forms= browser.get_forms()
form = forms[0]   # not strictly necessary since there is only ONE form in this page

print('')
print('This is the form - find here the input names: ' , form)
print('')
form['add_participant'] = 'RoboBOBO'
form['add_participant_checked_blue'] = 'a_yes__'  # To find what to write here use Firefox Inspector after clicking on an option.

browser.submit_form(form)



This is the form - find here the input names:  <RoboForm olduser=, add_participant=, add_participant_checked_blue=c_no___, add_participant_checked_green=c_no___>



# Micro exercise 3:
Visit the site https://dudle.inf.tu-dresden.de and create a new "Normal poll" at the address https://dudle.inf.tu-dresden.de/YOUR_NICKNAME. Then modify the code presented during the lecture to vote 3 times automatically (using a loop for example). **CAREFUL:** do not vote more times not to create trouble on the remote server!

This example proves that bots can interact massively with web interfaces without any human intervention.

Write your solution here:

In [30]:
import werkzeug
import string
werkzeug.cached_property = werkzeug.utils.cached_property  # Fix a bug in werkzeug.
from robobrowser import RoboBrowser
import random

def random_username():
    l = string.ascii_lowercase
    return ''.join(random.choice(l) for i in range(5))

def random_response():
    random_number = random.randint(0,1)
    if random_number == 0:
        return 'a_yes__'
    return 'c_no___'

for i in range(3):
    browser = RoboBrowser()
    browser.open('https://dudle.inf.tu-dresden.de/rick/')

    forms= browser.get_forms()
    form = forms[0]   # not strictly necessary since there is only ONE form in this page

    form['add_participant'] = random_username()
    form['add_participant_checked_cucumber'] = random_response()
    form['add_participant_checked_pickle'] = random_response()
    
    browser.submit_form(form)

# Micro exercise 4 (advanced and optional)
The site http://httpbin.org/ has been created by the same author of the module ```requests``` and it is an excellent site to test the functionality of the module.

For example, at the address:

http://httpbin.org/basic-auth/some_user/some_password (click on the link and log in with 'some_user' and 'some_password')

it is possible to test the authentication method with the chosen user name and password.

For more information watch this video at minute 10:
* Python Requests Tutorial: Request Web Pages, Download Images, POST Data, Read JSON, and More https://www.youtube.com/watch?v=tb8gHvYlCFs 

Run a few examples of your choice using the httpbin.org website.

# Multithreading with `concurrent.futures.ThreadPoolExecutor`

[Documentation Link](https://docs.python.org/3/library/concurrent.futures.html?highlight=concurrent%20future#module-concurrent.futures)

According to the documentation, "the `concurrent.futures` module provides a high-level interface for asynchronously executing callables". In this sentence, "asynchronously executing callables" means launching functions but do not wait for them to return before continuing. It provides a high-level way for multithreading.

Let's look at the following example:

In [32]:
from concurrent.futures import ThreadPoolExecutor

In [33]:
with ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(pow, 323, 1235)  # (1)
    print(future.result())  # (2)

7330187419711662525292446729952277967765833457839424373869116780517420149076198183898894905236339668910883614705749593461204605714622607490668683222808958179218191780471063516227460897872227090014097232715359988867409006240081206335670819048146328123935337644631341482039026277834498173918554303355637321060412263715606736911839847081166018722332660742474936263046482602637679145832497919840537694829188335160914131310111239449199642739655793719812086149415859534959085359215402107080568853413877372159233452025442228651418507639010743174496936173262981681095359156359401217962539764203947129055258900852300663811552683018727645219707243611502505332407412509113706415784954450373499498470564461122438759199401776852200401606640428677779377601753155808428527723343610817938497649317098807976560094452142608380040084257453294862872175833371967399756248792099979580669182692895291737594003884626270278670234491588018888596571060169103937561057223083861024046092685665503687387204434227037124219374130642

In the above example:

* line (1): starts a thread in the background which calculates the 1235th power of 323, but the main program continues!
* line (2): stops the main program until the future (i.e. the return value of pow()) is available.

So you submit callable objects (`pow` in the example above) to the `executor.submit()` function alongside its arguments. N.b.: You can also use `executor.map` for multiple sets of arguments.

There is also a `ProcessPoolExecutor` which starts a new process for every submission, instead of a new thread.

**Warning:** The usual multi-threading difficulties apply also here. Make sure you don't actually produce overhead by having to create threads/processes compared to your actual time of the task.

For a simple web-access, the networking time is usually the time-limiting factor and it could pay off to parallelize the network accesses. In the following example, we want to get the total size of the HTML of some webpages. For that, we are using the `requests` package.

In [34]:
import requests

pages = [
    'http://www.cnn.com/',
    'http://europe.wsj.com/',
    'http://www.bbc.co.uk/',
    'http://www.nzz.ch/',
    'http://some-non-existing-example.domain/'
]

def load_url(url, timeout):
    response = requests.get(url, timeout=timeout)
    return response.text

In [35]:
# Launch requests to pages in parallel
with ThreadPoolExecutor(max_workers=5) as executor:
    futures_to_url = {executor.submit(load_url, url, 10): url for url in pages}
    
# For every future, investigate result
for future, url in futures_to_url.items():
    try:
        data = future.result()
    except Exception as exc:
        print('%r generated an exception: %s' % (url, exc))
    else:
        print('%r page is %d bytes' % (url, len(data)))

'http://www.cnn.com/' page is 1131375 bytes
'http://europe.wsj.com/' page is 614935 bytes
'http://www.bbc.co.uk/' page is 282111 bytes
'http://www.nzz.ch/' page is 829468 bytes
'http://some-non-existing-example.domain/' generated an exception: HTTPConnectionPool(host='some-non-existing-example.domain', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x000001E736879F60>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


Let's compare the runtime between multi-threaded and serial execution.

In [41]:
%%timeit 

with ThreadPoolExecutor(max_workers=100) as executor:
    futures_to_url = {executor.submit(load_url, url, 10): url for url in pages}
    
for future, url in futures_to_url.items():
    try:
        data = future.result()
    except Exception as exc:
        #print('%r generated an exception: %s' % (url, exc)) # not printing for more precise time measurement
        pass
    else:
        #print('%r page is %d bytes' % (url, len(data)))
        pass

382 ms ± 35.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%%timeit

results = {}
for page in pages:
    try:
        data = load_url(page, 10)
    except:
        # non-existing-exmple.domain will raise error
        pass

1.13 s ± 93.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Depending on your OS and CPU configuration, the results will of course vary.

# Exercise 1: Find password of administrator page

Switch to the "Exercise_13.ipynb" notebook and solve exercise 1.

# SSH botnet
Most servers implement a simple security feature to prevent password-cracking attacks like the one above: they simply notice a too large number of requests from the same IP and block it for a certain amount of time. Fail2ban (https://www.fail2ban.org) is an example of program implementing such a feature.

A possible solution to this problem, is to automate connections from a large pool of different servers, a so-called *botnet*.

Python allows to run SSH connections therefore simplifying the construction of a large number of computers on the network controllable from a single terminal.

### Running SSH with the ```paramiko``` module
Some computers have not the ssh libraries installed and do not work as ssh servers. This is common for Windows computers, unless software such as Putty has been installed.

However, if the remote computer has Python and the module paramiko installed, it is possible to transform it to either a client or server for ssh connections. 

The example below is a client which connects to a remote server.

The `paramiko` module may have to be installed first. For example with `pip install paramiko`.

In [44]:
# The module paramiko does not require special installation:
# it is available in the folder paramiko inside the Lecture_13 folder.
# Best copy-paste this code to a separate file. Otherwise
# your NETHZ credentials could accidentially end up into the git repo.
import paramiko

ip = '129.132.3.158'  # tardis-c23.ee.ethz.ch is 129.132.3.158
def ssh_command(ip, user, pwd, command):
    client = paramiko.SSHClient()
    
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(ip, username = user, password = pwd)
    ssh_session = client.get_transport().open_session()
    if ssh_session.active:
        ssh_session.exec_command(command)
        print(ssh_session.recv(1024))
    return

command = 'ls -la'
# command = 'python3 write.py'  # notice that it is possible to launch python scripts too!
print("CLEAR THIS CELL BEFORE UPLOADING YOUR USERNAME AND PASSWORD TO THE GIT REPO!")

print("Enter your ETH username:")
username = input()

print("Enter your ETH password:")
password = input()

from IPython.display import clear_output
clear_output() # Wipe the output such that the password does not get printed there.

ssh_command(ip, username, password, command)


TimeoutError: [WinError 10060] Ein Verbindungsversuch ist fehlgeschlagen, da die Gegenstelle nach einer bestimmten Zeitspanne nicht richtig reagiert hat, oder die hergestellte Verbindung war fehlerhaft, da der verbundene Host nicht reagiert hat

### Domain name resolution
It is possible to use the domain name instead and let Python respolve the IP address:

In [45]:
import socket

domain_name = 'tardis-c23.ee.ethz.ch'

data = socket.gethostbyname_ex(domain_name)
print ("The representation of the answer is : ", repr(data) )  
print ("The IP Address of the Domain Name is: ", data[2][0])


The representation of the answer is :  ('tardis-c23.ee.ethz.ch', [], ['129.132.3.158'])
The IP Address of the Domain Name is:  129.132.3.158


### Running multiple commands

In `paramiko`, a session is a remote execution of a program. The program may be a shell, an application, a system command, or some built-in subsystem. Once the command has executed, the session is finished. Therefore, it is not possible to execute multiple commands in one session. What is possible, however, is to start a remote shell (which is *one* command), and interact with that shell through stdin etc. This is exemplified below:

In [ ]:

import paramiko
import socket

# Use socket to resolve the domain name into an IP address.
domain_name = 'tardis-c06.ee.ethz.ch'  # Choose a random XX = 01 to 38 here, do not overload a single PC!
data = socket.gethostbyname_ex(domain_name)
ip = data[2][0] 

def ssh_connect(ip, user, pwd):
    client = paramiko.SSHClient()    
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(ip, username = user, password = pwd)
    #ssh_session = client.get_transport().open_session()
    channel = client.invoke_shell()
    return client, channel

print("Enter your ETH username:")
username = input()

print("Enter your ETH password:")
password = input()

from IPython.display import clear_output
clear_output() # Wipe the output such that the password does not get printed there.

client, channel = ssh_connect(ip, username, password)

stdin = channel.makefile('wb')
stdout = channel.makefile('rb')

# Send multiple commands:
# the "exit" command seems important
stdin.write('''
id
ls
exit
''')

print(stdout.read())

# closing everything:
stdout.close()
stdin.close()
client.close()

### Micro exercise 5
The Python code below writes into a file the host name (like "tardis-c01") of the machine which is executing the code.

Write a code which logs into each of the 38 tardis computers and appends in the same file the current hostname.

Notice that all tardis-cXX computers share the **same network-attached drive**. Hence a file written on a remote machine will immediately be visible on the local machine as well.

In [ ]:
# Hint: Run this on each tardis machine.

import socket
import paramiko

def ssh_connect(ip, user, pwd):
        client = paramiko.SSHClient()    
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(ip, username = user, password = pwd)
        #ssh_session = client.get_transport().open_session()
        channel = client.invoke_shell()
        return client, channel

print("Enter your ETH username:")
username = input()
print("Enter your ETH password:")
password = input()
from IPython.display import clear_output
clear_output() # Wipe the output such that the password does not get printed there.

for i in range(38):
    if (i<10):
        computer_number = '0'+ str(i+1) 
    else:
        computer_number = str(i+1)
        
    # Use socket to resolve the domain name into an IP address.
    domain_name = 'tardis-c'+computer_number+'.ee.ethz.ch'  # Choose a random XX = 01 to 38 here, do not overload a single PC!
    data = socket.gethostbyname_ex(domain_name)
    ip = data[2][0] 

    client, channel = ssh_connect(ip, username, password)

    hostname = socket.gethostname() # does that line still work?
    with open('bot-micro-exercise.txt', 'a') as f:
        f.write(hostname)
        f.write('\n')

f.close() #doesn t with host name already close the file automatically?

Enter your ETH username:


# Exercise 2: Simulate a DDoS attack
Switch to the "Exercise_13.ipynb" notebook and solve exercise 2.